In [1]:
dataset_name = "W14_comb"
df_list = [
           "BES_Panel",
           "BES_reduced_with_na",
#            "BESnumeric"
#            "BES_reduced",
#            "BESnumeric"
          ]

In [2]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
import pickle, os, gc, re
sns.set()
sns.palplot(sns.color_palette("colorblind"))
from IPython.display import display, display_html, HTML

import Jupyter_module_loader
from utility import *
import gaussian_kde


In [3]:
# you should clone this git to a subdirectory called 'BES_analysis_code' (in some directory - I call it BES_analysis - doesn't matter though)
%matplotlib inline
encoding = "ISO-8859-1"

(BES_code_folder, BES_small_data_files, BES_data_folder,
 BES_output_folder, BES_file_manifest, BES_R_data_files) = setup_directories()




In [4]:
global BES_Panel, BES_numeric, BES_reduced, BES_reduced_with_na, BES_non_numeric
data_subfolder = BES_data_folder + dataset_name + os.sep

(manifest, dataset_filename, dataset_description, dataset_citation,
 dataset_start, dataset_stop, dataset_wave) = get_manifest(dataset_name, BES_file_manifest)

for df in df_list:
    if df=="BES_Panel":
        globals()[df]  = pd.read_msgpack(data_subfolder + dataset_filename.replace('.dta','.msgpack'))
    else:
        globals()[df]  = pd.read_msgpack(data_subfolder + df + '.msgpack' )
        globals()[df].replace(-1,np.nan,inplace=True)
  
(var_type, cat_dictionary, new_old_col_names, old_new_col_names) = get_small_files(data_subfolder, encoding)

var_type (5692, 13)


In [6]:
# pan_dataset_ages = pd.read_msgpack(    BES_small_data_files + "pan_dataset_ages"+".msgpack" )
pan_dataset_allr_values = pd.read_csv( BES_small_data_files + "pan_dataset_allr_values"+".csv" )

In [9]:
pan_dataset_allr_values.columns

Index(['Economic: Left(0) <-> Right(200){uniform}',
       'Social: Liberalism(0) <-> Conservatism(200){uniform}',
       'Economic: Left <-> Right{gaussian-ish}',
       'Social: Liberalism <-> Conservatism{gaussian-ish}', 'id'],
      dtype='object')

In [13]:
corrs = BES_reduced_with_na.corrwith( pan_dataset_allr_values[pan_dataset_allr_values.columns[3]] )

In [17]:
corrs.sort_values().tail(100)[0:70]

changeImmigW3__Getting a lot higher               0.489178
blackEqualityW6_W12__Gone much too far            0.489371
leftRightW9__Right                                0.489485
euRefVoteW4_Leave the EU                          0.489956
euRefVoteUnsqueezeW8_Leave the EU                 0.490231
euIDW8_The leave side                             0.491403
EUIntegrationSelfW9__Protect our independence     0.491637
changeImmigW7__Getting a lot higher               0.492255
govtHandoutsW2__Strongly agree                    0.492763
EUIntegrationSelfW10__Protect our independence    0.492991
leftRightW12__Right                               0.493420
euRefVoteW6_Leave the EU                          0.494043
handleMIIUKIPW9__Very well                        0.494828
leftRightW11__Right                               0.494882
euIDW7_The leave side                             0.495115
auth1W11_Respect for elders                       0.495585
happyLeaveW7__Very happy                          0.4956

In [26]:
search(BES_reduced_with_na,"govtHandout")

govtHandoutsW1__Strongly agree    29525
govtHandoutsW2__Strongly agree    29461
govtHandoutsW3__Strongly agree    27015
govtHandoutsW4__Strongly agree    30628
govtHandoutsW7__Strongly agree    29375
dtype: int64

In [28]:
BES_reduced_with_na["likeUKIPW1__Strongly like"].corr( BES_reduced_with_na["govtHandoutsW1__Strongly agree"] )

0.35340844977572256

In [29]:
BES_reduced_with_na["likeUKIPW2__Strongly like"].corr( BES_reduced_with_na["govtHandoutsW2__Strongly agree"] )

0.3666052106839375

In [30]:
BES_reduced_with_na["likeUKIPW3__Strongly like"].corr( BES_reduced_with_na["govtHandoutsW3__Strongly agree"] )

0.37568163805818333

In [31]:
BES_reduced_with_na["likeUKIPW4__Strongly like"].corr( BES_reduced_with_na["govtHandoutsW4__Strongly agree"] )

0.3468223514843521

In [32]:
BES_reduced_with_na["likeUKIPW7__Strongly like"].corr( BES_reduced_with_na["govtHandoutsW7__Strongly agree"] )

0.36972526848975906

In [37]:
BES_reduced_with_na[search(BES_reduced_with_na,"like.*W1__Strongly like").index].corrwith( BES_reduced_with_na["govtHandoutsW1__Strongly agree"] )

likeCameronW1__Strongly like     0.381053
likeMilibandW1__Strongly like   -0.335237
likeCleggW1__Strongly like       0.081406
likeSalmondW1__Strongly like    -0.256044
likeFarageW1__Strongly like      0.310298
likeConW1__Strongly like         0.425824
likeLabW1__Strongly like        -0.321300
likeLDW1__Strongly like          0.020238
likeUKIPW1__Strongly like        0.353408
likeGrnW1__Strongly like        -0.317548
likeBNPW1__Strongly like         0.183336
dtype: float64

In [24]:
corrs.loc[ [x for x in corrs.index if re.match("auth(\d)", x)] ].sort_values()

auth2W10_Self-reliance        -0.433385
auth2W11_Self-reliance        -0.417702
auth2W7_Self-reliance         -0.414478
auth2W14_Self-reliance        -0.339960
auth3W14_To be well-behaved    0.257265
auth3W10_To be well-behaved    0.292794
auth3W7_To be well-behaved     0.312503
auth3W11_To be well-behaved    0.314702
auth4W14_Good manners          0.402783
auth4W7_Good manners           0.427815
auth1W14_Respect for elders    0.429728
auth4W11_Good manners          0.432644
auth4W10_Good manners          0.439897
auth1W7_Respect for elders     0.484471
auth1W11_Respect for elders    0.495585
auth1W10_Respect for elders    0.518273
dtype: float64

In [25]:
corrs.loc[ [x for x in corrs.index if re.match("authL", x)] ].sort_values()

authLong4W10__Strongly agree   -0.399055
authLong1W10__Strongly agree   -0.376369
authLong3W10__Strongly agree   -0.233478
authLong2W10__Strongly agree    0.512047
dtype: float64

In [24]:
# globals()["BES_Panel"]  = pd.read_stata(data_subfolder + dataset_filename)

In [25]:
# data_subfolder

In [26]:
# BES_Panel.to_msgpack(data_subfolder + "BES2019_W15_Panel_v0.3.msgpack")

In [27]:
# BES_data_folder

In [13]:
# BES_PanelW15 = pd.read_stata(BES_data_folder + "W15_only"+os.sep + "BES2019_W15_v0.2.dta")

In [14]:
# BES_PanelW15.to_msgpack(BES_data_folder + "W15_only"+os.sep + "BES2019_W15_v0.2.msgpack")

In [ ]:
# BES_PanelW15 = pd.read_stata(BES_data_folder + "W15_only"+os.sep + "BES2019_W15_v0.2.dta")

In [28]:
BES_PanelW15 = pd.read_msgpack(BES_data_folder + "W15_only"+os.sep + "BES2019_W15_v0.2.msgpack")

In [29]:
search(BES_PanelW15,"strongLeader")

strongLeader    30842
dtype: int64

In [37]:
BES_reduced_with_na[search(BES_reduced_with_na,"strongLeader").index].diff()

,strongLeaderW10__Strongly agree,strongLeaderW11__Strongly agree,strongLeaderW14__Strongly agree,strongLeaderW15__Strongly agree
0,NaN,NaN,NaN,NaN
1,0.0,NaN,NaN,NaN
2,0.0,-1.0,-1.0,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,NaN,2.0,NaN,NaN
7,1.0,1.0,0.0,-1.0
8,-3.0,-3.0,-2.0,0.0
9,NaN,NaN,NaN,-1.0


In [34]:
BES_reduced_with_na["strongLeaderW15__Strongly agree"] = BES_PanelW15["strongLeader"].cat.codes.replace(-1,np.nan)

In [ ]:
#         immig.columns = ["Delta_"+x for x in immig.columns]
#         if num ==0:
#             immigDiff = immig.diff(axis=1).drop(immig.columns[0],axis=1)
#         else:
#             immigDiff = pd.concat( [immigDiff, immig.diff(axis=1).drop(immig.columns[0],axis=1)], axis=1)

In [38]:
diffs = BES_reduced_with_na[search(BES_reduced_with_na,"strongLeader").index].diff(axis=1)


In [61]:
diffW14_W15 = BES_reduced_with_na["strongLeaderW15__Strongly agree"] - BES_reduced_with_na["strongLeaderW14__Strongly agree"]
diffW14_W15.notnull().sum()

15086

In [62]:
corrs = BES_reduced_with_na.corrwith(diffW14_W15)
# corrs

ageW1                                                 -0.023081
turnoutUKGeneralW1__Very likely that I will vote      -0.027770
turnoutUKGeneralW2__Very likely that I will vote      -0.026766
turnoutUKGeneralW3__Very likely that I will vote      -0.030392
turnoutUKGeneralW4__Very likely that I will vote      -0.010683
turnoutUKGeneralW5__Very likely that I will vote      -0.007528
turnoutUKGeneralW7__Very likely that I will vote      -0.002602
turnoutUKGeneralW8__Very likely that I will vote       0.023761
turnoutUKGeneralW9__Very likely that I will vote       0.011989
turnoutUKGeneralW10__Very likely that I will vote      0.015013
turnoutUKGeneralW11__Very likely that I will vote      0.005730
turnoutUKGeneralW12__Very likely that I will vote      0.031325
generalElectionCertaintyW1__Completely certain         0.005640
generalElectionCertaintyW2__Completely certain        -0.013310
generalElectionCertaintyW3__Completely certain        -0.003668
generalElectionCertaintyW4__Completely c

In [63]:
corrs.sort_values().dropna()

strongLeaderW14__Strongly agree                                                                             -0.476722
al_scaleW14__Authoritarian                                                                                  -0.179497
antiIntellectualW8__Strongly agree                                                                          -0.166995
strongLeaderW10__Strongly agree                                                                             -0.164698
blackEquality2W14__Gone much too far                                                                        -0.164235
ptvUKIPW8__Very likely                                                                                      -0.162917
overseasAidW14__Strongly agree                                                                              -0.160303
al2W14__Strongly agree                                                                                      -0.155169
al_scaleW10W11W12__Authoritarian                        